# `CLOUD.jl` - 2D linear advection example

In [1]:
using OrdinaryDiffEq
using CLOUD;

[ Info: Installing pyqt package to avoid buggy tkagg backend.
┌ Warning: No working GUI backend found for matplotlib
└ @ PyPlot ~/.julia/packages/PyPlot/XaELc/src/init.jl:165


┌ Info: Installing pyqt package to avoid buggy tkagg backend.
└ @ PyPlot /Users/tristanmontoya/.julia/packages/PyPlot/XaELc/src/init.jl:118


Define the physical problem

In [2]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period
initial_data = InitialDataSine(A,k)
conservation_law = linear_advection_equation((a*cos(θ),a*sin(θ)),λ=1.0);

Set discretization parameters

In [3]:
M = 4
p = 4
p_map = 4
form = SplitConservationForm()
integrator = CarpenterKennedy2N54()
dt_scale=0.05
dt = dt_scale*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [4]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

visualize(spatial_discretization, results_path, "mesh.pdf", 
    plot_volume_nodes=false, grid_lines=true)

Run the solver and plot the solution

In [5]:
ode_problem = semidiscretize(load_project(results_path)...)
save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, integrator, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")
plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf", u_range=[-1.0,1.0], contours=25,
    label="U^h(\\mathbf{x},t)")


 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:           451ms / 1091.2%          57.4MiB / 5287.5%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          81.5k    2.16s   43.9%  26.5μs   0.98GiB   32.9%  12.6KiB
   volume terms         81.5k    1.37s   27.9%  16.8μs    674MiB   22.2%  8.47KiB
   facet terms          81.5k    503ms   10.2%  6.16μs    212MiB    7.0%  2.66KiB
   mass matrix solve    81.5k   94.9ms    1.9%  1.16μs   39.8MiB    1.3%     512B
 eval flux diff         81.5k    1.41s   28.5%  17.2μs   1.11GiB   37.6%  14.3KiB
 extrapolate solution   81.5k    681ms   13.8%  8.35μs    269MiB    8.9%  3.38KiB
 eval numerical

In [6]:
visualize(initial_data, plotter, "exact.pdf", u_range=[-1.0,1.0], contours=17, label="U^0(\\mathbf{x})")

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [7]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:


0.000683954884588324
Conservation (initial/final/diff):


[-1.108594540819019e-8][-1.1085945919933615e-8][-5.117434254131581e-16]
Energy (initial/final/diff):


[0.12499998811286066][0.12497956180878655][-2.0426304074103174e-5]


Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [8]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

visualize(spatial_discretization, results_path, "mesh.pdf", 
    plot_volume_nodes=true, grid_lines=false)

Run the solver and plot the solution

In [9]:
ode_problem = semidiscretize(load_project(results_path)...)

save_solution(ode_problem.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem, integrator, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf", u_range=[-1.0,1.0], contours=25,
    label="U^h(\\mathbf{x},t)")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:         97.4μs / 5841714.9%      12.9KiB / 40468359.8%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual           163k    3.00s   52.7%  18.4μs   2.44GiB   49.0%  15.7KiB
   volume terms          163k    2.16s   37.9%  13.2μs   1.95GiB   39.0%  12.5KiB
   mass matrix solve     163k    443ms    7.8%  2.72μs    371MiB    7.3%  2.33KiB
   facet terms           163k    152ms    2.7%   933ns   39.0MiB    0.8%     251B
 eval flux diff          163k    1.67s   29.3%  10.2μs   1.34GiB   26.9%  8.61KiB
 eval flux               163k    322ms    5.7%  1.97μs    139MiB    2.7%     896B
 eval numerical

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [11]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0034367122388651375
Conservation (initial/final/diff):


[-1.4530229013823975e-7][-1.4530229025620095e-7][-1.1796119636642288e-16]
Energy (initial/final/diff):
[0.12499475017905026][0.12485504074741147][-0.0001397094316387959]
